In [1]:
import os
os.chdir("../")
%pwd

'c:\\Users\\riswa\\Desktop\\AI\\Text-Summarizer'

# DAta Validation


In [3]:
from dataclasses import dataclass
from pathlib import  Path
from src.summarizer.constants import *

@dataclass(frozen= True)

class DataValidationConfig:
    root_dir : Path
    status_file: str
    all_required_files: list


In [4]:
from src.summarizer.logging import logging
from src.summarizer.exception import CustomException
from dataclasses import dataclass
from pathlib import Path
from src.summarizer.constants import  *
from src.summarizer.utils import *


# config Manager
|

In [11]:
class ConfigurationManager:

    def __init__(self,config_path = CONFIG_PATH,params_path = PARAMS_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) ->DataValidationConfig:
        config = self.config.data_validation
        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            status_file = config.status_file,
            all_required_files = config.all_required_files
        )

        return data_validation_config
    

# Components

In [12]:
import os
import urllib.request as request
import zipfile
from summarizer.logging import logger
from summarizer.utils  import get_size
import sys

In [16]:
class DataValidation:
    def __init__(self,config : DataValidationConfig):
        self.config = config

    def validate_all_files_exist(self)-> bool:
        try:
            validation_status = None

            all_files = os.listdir(os.path.join("artifacts","data_ingestion","samsum_dataset"))
            for file in all_files:
                if file not in self.config.all_required_files:
                    validation_status = False
                    with open(self.config.status_file,"w") as f:
                        f.write(f"Validation status: {validation_status}")

                else: 
                    validation_status = True
                    with open(self.config.status_file,"r") as f:
                        existing_status = f.read()
                        if existing_status == "Validation status: True":
                            logger.info("No new files found. Skipping validation")
                        else:
                            with open(self.config.status_file,"w") as f:
                                f.write(f"Validation status: {validation_status}")
        except Exception    as e:
                raise CustomException(e,sys)


In [17]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation =  DataValidation(config=data_validation_config)
    data_validation.validate_all_files_exist()
except Exception as e:
    raise e

[2024-08-15 20:23:37,763: INFO: __init__: yaml file: config\config.yaml loaded successfully]
[2024-08-15 20:23:37,767: INFO: __init__: yaml file: params.yaml loaded successfully]
[2024-08-15 20:23:37,770: INFO: __init__: created directory at: artifacts]
[2024-08-15 20:23:37,772: INFO: __init__: created directory at: artifacts/data_validation]
[2024-08-15 20:23:38,420: INFO: __init__: No new files found. Skipping validation]
[2024-08-15 20:23:38,424: INFO: __init__: No new files found. Skipping validation]
